In [1]:
from collections import Counter, deque
from copy import deepcopy
from itertools import combinations
from random import sample
from typing import TypeAlias

Graph: TypeAlias = dict[str, set[str]]
Edge: TypeAlias = tuple[str, str]

In [2]:
def process_line(line: str) -> tuple[str, set[str]]:
    parts = line.strip().split(": ")
    return parts[0], set(parts[1].split())


def make_bidirectional(graph: Graph) -> Graph:
    bidirection_graph = deepcopy(graph)
    for node, neighbors in graph.items():
        for neighbor in neighbors:
            if neighbor not in bidirection_graph:
                bidirection_graph[neighbor] = {node}
            else:
                bidirection_graph[neighbor].add(node)
    return bidirection_graph

In [3]:
def cluster_size(graph: Graph, seed: str) -> int:
    visited: set[str] = {seed}
    to_visit: deque[str] = deque(graph[seed])

    while len(to_visit):
        # dfs or bfs does not matter here
        current = to_visit.pop()
        visited.add(current)
        to_visit.extend((i for i in graph[current] if i not in visited))

    return len(visited)


def backtrack(prev: dict[str, str], start: str, end: str) -> list[Edge]:
    trace: list[Edge] = []
    current = end

    while current != start:
        trace.append(tuple(sorted([current, prev[current]])))
        current = prev[current]

    return trace[::-1]


def get_path(graph: Graph, start: str, end: str) -> list[Edge]:
    """Use BFS and backtracking to get the shortest path between nodes."""
    if end == start:
        return []

    prev: dict[str, str] = {neighbor: start for neighbor in graph[start]}
    prev[start] = start
    to_visit: deque[str] = deque(graph[start])

    while len(to_visit):
        current = to_visit.popleft()
        if current == end:
            return backtrack(prev, start, end)
        for neighbor in graph[current]:
            if neighbor not in prev:
                prev[neighbor] = current
                to_visit.append(neighbor)

    return []

In [4]:
with open("input25.txt", "r") as f:
    graph = dict(map(process_line, f))
    bidirectional_graph = make_bidirectional(graph)

In [5]:
num_traversals = Counter()

for start, end in sample(list(combinations(bidirectional_graph.keys(), 2)), 1000):
    num_traversals.update(get_path(bidirectional_graph, start, end))

num_traversals.most_common(3)

[(('brd', 'clb'), 256), (('bbz', 'jxd'), 132), (('glz', 'mxd'), 122)]

In [6]:
for (start, end), _ in num_traversals.most_common(3):
    bidirectional_graph[start].remove(end)
    bidirectional_graph[end].remove(start)

In [7]:
cluster_size(bidirectional_graph, "brd") * cluster_size(bidirectional_graph, "clb")

518391